## Sensitive analysis:


In [2]:
from sast.sast import *
from sktime.datasets import load_UCR_UEA_dataset, tsc_dataset_names
from sktime.classification.kernel_based import RocketClassifier
import time
import pandas as pd


### Select Datasets for hypertunning RSAST

It is runned RSAST in a set of UCR datasets with a predefined number of runs ("runs"). Then, it is selected a range ("range_total") between 1, 10, 100 and 10000 for the selected dataset.

In [3]:
ds_sens = tsc_dataset_names.univariate_equal_length
#ds_sens = ['ACSF1', 'Adiac', 'ArrowHead', 'Beef', 'BeetleFly', 'BirdChicken', 'BME', 'Car', 'CBF', 'Chinatown', 'ChlorineConcentration', 'CinCECGTorso', 'Coffee']
ds_sens = ['Chinatown']
max_ds=len(ds_sens) #exploring dataset in UEA & UCR Time Series Classification Repository
#print(ds_sens)
print(ds_sens)


['Chinatown']


In [4]:
runs = 3
range_total = [1,10,30,50,100]
#list
df_result = {}
list_score = []
list_overall_time = []
list_cweight_time = []
list_fsubsequence_time = []
list_tdataset_time = []
list_tclassifier_time = []
list_dataset = []
list_hyperparameter = []
list_method = []
list_rpoint = []
list_nb_per_class = []
list_len_method = []

for ds in ds_sens:
    print("ds="+ds)
    X_train, y_train = load_UCR_UEA_dataset(name=ds, extract_path='data', split="train", return_type="numpy2d")
    X_test, y_test = load_UCR_UEA_dataset(name=ds, extract_path='data', split="test", return_type="numpy2d")
    for i in range(runs):
        for p in range_total:
            for g in range_total:
                if p ==0:
                    p=1
                if g ==0:
                    g=1
                print("ACF&PACF: n_random_points="+str(p)+" nb_inst_per_class="+str(g))
                start = time.time()
                random_state = None
                rsast_ridge = RSAST(n_random_points=p,nb_inst_per_class=g, len_method="both",classifier=RidgeClassifierCV())
                rsast_ridge.fit(X_train, y_train)
                end = time.time()
                list_score.append(rsast_ridge.score(X_test,y_test))

                list_overall_time.append(end-start)
                list_cweight_time.append(rsast_ridge.time_calculating_weights)
                list_fsubsequence_time.append(rsast_ridge.time_creating_subsequences)
                list_tdataset_time.append(rsast_ridge.transform_dataset)
                list_tclassifier_time.append(rsast_ridge.time_classifier)

                list_dataset.append(ds)
                list_hyperparameter.append("ACF&PACF: n_random_points="+str(p)+" nb_inst_per_class="+str(g))
                list_rpoint.append(str(p))
                list_nb_per_class.append(str(g))
                list_method.append("Rsast")
                list_len_method.append("ACF&PACF")
                
                print("ACF: n_random_points="+str(p)+" nb_inst_per_class="+str(g))
                start = time.time()
                random_state = None
                rsast_ridge = RSAST(n_random_points=p,nb_inst_per_class=g, len_method="ACF",classifier=RidgeClassifierCV())
                rsast_ridge.fit(X_train, y_train)
                end = time.time()
                list_score.append(rsast_ridge.score(X_test,y_test))
                
                list_overall_time.append(end-start)
                list_cweight_time.append(rsast_ridge.time_calculating_weights)
                list_fsubsequence_time.append(rsast_ridge.time_creating_subsequences)
                list_tdataset_time.append(rsast_ridge.transform_dataset)
                list_tclassifier_time.append(rsast_ridge.time_classifier)

                list_dataset.append(ds)
                list_hyperparameter.append("ACF: n_random_points="+str(p)+" nb_inst_per_class="+str(g))
                list_rpoint.append(str(p))
                list_nb_per_class.append(str(g))
                list_method.append("Rsast")
                list_len_method.append("ACF")
                
                print("PACF: n_random_points="+str(p)+" nb_inst_per_class="+str(g))
                start = time.time()
                random_state = None
                rsast_ridge = RSAST(n_random_points=p,nb_inst_per_class=g, len_method="PACF",classifier=RidgeClassifierCV())
                rsast_ridge.fit(X_train, y_train)
                end = time.time()
                list_score.append(rsast_ridge.score(X_test,y_test))
                
                list_overall_time.append(end-start)
                list_cweight_time.append(rsast_ridge.time_calculating_weights)
                list_fsubsequence_time.append(rsast_ridge.time_creating_subsequences)
                list_tdataset_time.append(rsast_ridge.transform_dataset)
                list_tclassifier_time.append(rsast_ridge.time_classifier)

                list_dataset.append(ds)
                list_hyperparameter.append("PACF: n_random_points="+str(p)+" nb_inst_per_class="+str(g))
                list_rpoint.append(str(p))
                list_nb_per_class.append(str(g))
                list_method.append("Rsast")
                list_len_method.append("PACF")
                
                print("Max ACF: n_random_points="+str(p)+" nb_inst_per_class="+str(g))
                start = time.time()
                random_state = None
                rsast_ridge = RSAST(n_random_points=p,nb_inst_per_class=g, len_method="Max ACF",classifier=RidgeClassifierCV())
                rsast_ridge.fit(X_train, y_train)
                end = time.time()
                list_score.append(rsast_ridge.score(X_test,y_test))
                
                list_overall_time.append(end-start)
                list_cweight_time.append(rsast_ridge.time_calculating_weights)
                list_fsubsequence_time.append(rsast_ridge.time_creating_subsequences)
                list_tdataset_time.append(rsast_ridge.transform_dataset)
                list_tclassifier_time.append(rsast_ridge.time_classifier)
                
                list_dataset.append(ds)
                list_hyperparameter.append("Max ACF: n_random_points="+str(p)+" nb_inst_per_class="+str(g))
                list_rpoint.append(str(p))
                list_nb_per_class.append(str(g))
                list_method.append("Rsast")
                list_len_method.append("Max ACF")

                print("Max PACF: n_random_points="+str(p)+" nb_inst_per_class="+str(g))
                start = time.time()
                random_state = None
                rsast_ridge = RSAST(n_random_points=p,nb_inst_per_class=g, len_method="Max PACF",classifier=RidgeClassifierCV())
                rsast_ridge.fit(X_train, y_train)
                end = time.time()
                list_score.append(rsast_ridge.score(X_test,y_test))
                
                list_overall_time.append(end-start)
                list_cweight_time.append(rsast_ridge.time_calculating_weights)
                list_fsubsequence_time.append(rsast_ridge.time_creating_subsequences)
                list_tdataset_time.append(rsast_ridge.transform_dataset)
                list_tclassifier_time.append(rsast_ridge.time_classifier)
                
                list_dataset.append(ds)
                list_hyperparameter.append("Max PACF: n_random_points="+str(p)+" nb_inst_per_class="+str(g))
                list_rpoint.append(str(p))
                list_nb_per_class.append(str(g))
                list_method.append("Rsast")
                list_len_method.append("Max PACF")

                print("None: n_random_points="+str(p)+" nb_inst_per_class="+str(g))
                start = time.time()
                random_state = None
                rsast_ridge = RSAST(n_random_points=p,nb_inst_per_class=g, len_method="None",classifier=RidgeClassifierCV())
                rsast_ridge.fit(X_train, y_train)
                end = time.time()
                list_score.append(rsast_ridge.score(X_test,y_test))
                
                list_overall_time.append(end-start)
                list_cweight_time.append(rsast_ridge.time_calculating_weights)
                list_fsubsequence_time.append(rsast_ridge.time_creating_subsequences)
                list_tdataset_time.append(rsast_ridge.transform_dataset)
                list_tclassifier_time.append(rsast_ridge.time_classifier)
                
                list_dataset.append(ds)
                list_hyperparameter.append("None: n_random_points="+str(p)+" nb_inst_per_class="+str(g))
                list_rpoint.append(str(p))
                list_nb_per_class.append(str(g))
                list_method.append("Rsast")
                list_len_method.append("None")
                

df_result['accuracy']=list_score

df_result['time']=list_overall_time
df_result['cweights_time']=list_cweight_time
df_result['fsubsequence_time']=list_fsubsequence_time
df_result['tdataset_time']=list_tdataset_time
df_result['tclassifier_time']=list_tclassifier_time


df_result['dataset_name']=list_dataset
df_result['classifier_name']=list_hyperparameter
df_result['rpoint']=list_rpoint
df_result['nb_per_class']=list_nb_per_class
df_result['method']=list_method
df_result['len_method']=list_len_method

df_result=pd.DataFrame(df_result)
# export a overall dataset with results
df_result.to_csv("df_overall_tunning.csv") 

ds=Chinatown
ACF&PACF: n_random_points=1 nb_inst_per_class=1
ACF: n_random_points=1 nb_inst_per_class=1
PACF: n_random_points=1 nb_inst_per_class=1
Max ACF: n_random_points=1 nb_inst_per_class=1
Max PACF: n_random_points=1 nb_inst_per_class=1
None: n_random_points=1 nb_inst_per_class=1
ACF&PACF: n_random_points=1 nb_inst_per_class=10
ACF: n_random_points=1 nb_inst_per_class=10
PACF: n_random_points=1 nb_inst_per_class=10
Max ACF: n_random_points=1 nb_inst_per_class=10
Max PACF: n_random_points=1 nb_inst_per_class=10
None: n_random_points=1 nb_inst_per_class=10
ACF&PACF: n_random_points=1 nb_inst_per_class=30
ACF: n_random_points=1 nb_inst_per_class=30
PACF: n_random_points=1 nb_inst_per_class=30
Max ACF: n_random_points=1 nb_inst_per_class=30
Max PACF: n_random_points=1 nb_inst_per_class=30
None: n_random_points=1 nb_inst_per_class=30
ACF&PACF: n_random_points=1 nb_inst_per_class=50
ACF: n_random_points=1 nb_inst_per_class=50
PACF: n_random_points=1 nb_inst_per_class=50
Max ACF: n_rand

In [5]:
df_result=pd.read_csv("df_overall_tunning.csv")

In [6]:
# create and export a pivot table with the mean of score
pivot_table = pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
pivot_table.to_csv("df_pivot_tunning_hyper.csv") 


In [7]:
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class','rpoint'], aggfunc='mean')

nb_per_class       1                                                 10   \
rpoint             1         10        30        50        100       1     
len_method                                                                 
ACF           0.806608  0.848397  0.869776  0.932945  0.905734  0.965015   
ACF&PACF      0.837707  0.918367  0.940719  0.932945  0.935860  0.938776   
Max ACF       0.619048  0.849368  0.929057  0.796890  0.906706  0.941691   
Max PACF      0.835763  0.926142  0.964043  0.943635  0.949466  0.932945   
None          0.787172  0.923226  0.951409  0.896016  0.960155  0.931973   
PACF          0.824101  0.924198  0.949466  0.941691  0.945578  0.926142   

nb_per_class                                          ...       50             \
rpoint             10        30        50        100  ...       1         10    
len_method                                            ...                       
ACF           0.970845  0.975705  0.981535  0.980564  ...  0.931001  0.971817   
ACF&PACF      0.978620  0.983479  0.984451  0.983479  ...  0.959184  0.975705   
Max ACF       0.969874  0.974733  0.971817  0.976676  ...  0.968902  0.966958   
Max PACF      0.974733  0.983479  0.984451  0.984451  ...  0.929057  0.982507   
None          0.974733  0.977648  0.966958  0.979592  ...  0.937804  0.968902   
PACF          0.978620  0.983479  0.982507  0.981535  ...  0.950437  0.972789   

nb_per_class                                     100                      \
rpoint             30        50        100       1         10        30    
len_method                                                                 
ACF           0.969874  0.977648  0.979592  0.963071  0.967930  0.969874   
ACF&PACF      0.981535  0.983479  0.983479  0.937804  0.982507  0.979592   
Max ACF       0.978620  0.979592  0.969874  0.919339  0.974733  0.979592   
Max PACF      0.981535  0.983479  0.984451  0.939747  0.980564  0.984451   
None          0.976676  0.974733  0.976676  0.959184  0.979592  0.978620   
PACF          0.984451  0.983479  0.982507  0.909621  0.975705  0.981535   

nb_per_class                      
rpoint             50        100  
len_method                        
ACF           0.976676  0.965015  
ACF&PACF      0.982507  0.982507  
Max ACF       0.965015  0.970845  
Max PACF      0.983479  0.983479  
None          0.972789  0.979592  
PACF          0.982507  0.984451  

[6 rows x 25 columns]

In [8]:
# create a pivot table with the mean of score by hyperparameter
pd.pivot_table(df_result, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.785066,0.939423,0.929219,0.946064,0.938128
10,0.898283,0.974571,0.976676,0.973113,0.976838
30,0.934078,0.979754,0.979592,0.978782,0.978944
50,0.907353,0.978620,0.978296,0.980402,0.977162
100,0.933916,0.981050,0.980726,0.979430,0.977648


In [9]:
# filter by method with ACF&PACF
# create a pivot table with the mean of score by hyperparameter
df_result_acc=df_result[df_result["len_method"]=="ACF&PACF"]
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.837707,0.938776,0.935860,0.959184,0.937804
10,0.918367,0.978620,0.978620,0.975705,0.982507
30,0.940719,0.983479,0.979592,0.981535,0.979592
50,0.932945,0.984451,0.984451,0.983479,0.982507
100,0.935860,0.983479,0.982507,0.983479,0.982507


In [10]:
# create a pivot table with the variatioon of score by hyperparameter
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.019655,0.000773,0.001462,0.000025,0.000853
10,0.002830,0.000003,0.000054,0.000037,0.000008
30,0.000394,0.000011,0.000000,0.000020,0.000008
50,0.000535,0.000003,0.000003,0.000003,0.000008
100,0.001028,0.000003,0.000008,0.000003,0.000008


#### Analysis of time complexity

##### Overall time complexity

In [10]:
# create a pivot table with the mean of time spent in minutes
pd.pivot_table(df_result_acc, values='time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,3.593273,0.045050,0.045838,0.050864,0.045087
10,0.020180,0.116662,0.116545,0.125694,0.114510
30,0.038457,0.268633,0.266252,0.323794,0.266086
50,0.068816,0.432791,0.437796,0.430851,0.430789
100,0.097596,0.837758,0.828782,0.857705,0.965418


##### Calculate weights time complexity

In [11]:
pd.pivot_table(df_result_acc, values='cweights_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.009942,0.005874,0.007020,0.007980,0.007199
10,0.005986,0.006852,0.007979,0.009974,0.007979
30,0.007726,0.005983,0.005992,0.010971,0.006017
50,0.006983,0.006981,0.005977,0.007017,0.007978
100,0.006981,0.006981,0.006016,0.006980,0.013963


##### Finding subsequences time complexity

In [40]:
pd.pivot_table(df_result_acc, values='fsubsequence_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,100
rpoint,,,
1,0.005742,0.030302,0.036330
10,0.008799,0.053819,0.055026
100,0.025177,0.233774,0.230257


##### Transform Dataset time complexity

In [41]:
pd.pivot_table(df_result_acc, values='tdataset_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,100
rpoint,,,
1,0.021197,0.012470,0.012308
10,0.020159,0.066478,0.080071
100,0.075172,0.659409,0.659667


##### Classifier time complexity

In [42]:
pd.pivot_table(df_result_acc, values='tclassifier_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,100
rpoint,,,
1,0.001901,0.015479,0.009955
10,0.009999,0.003813,0.008056
100,0.005707,0.003417,0.007890
